# Johnson Counter

Original design by MyHDL authors.
See https://www.myhdl.org/docs/examples/jc2.html.

The purpose of this example is to compare the styles of MyHDL and `SeqiLog`.

In [ ]:
import sys
from pathlib import Path

WORKSPACE = Path("..")

sys.path.insert(0, str(WORKSPACE / "src"))

In [ ]:
from bvwx import Enum, Vec, cat
from vcd import VCDWriter

from deltacycle import run

from seqlogic import ITE, Module
from seqlogic.control.sync import drv_clock, drv_reset

In [ ]:
class Direction(Enum):
    LEFT = "1b0"
    RIGHT = "1b1"


class JohnsonCounter(Module):
    N: int = 4

    def build(self):
        clk = self.input(name="clk", dtype=Vec[1])
        rst = self.input(name="rst", dtype=Vec[1])

        go = self.input(name="go", dtype=Vec[1])
        direction = self.input(name="direction", dtype=Direction)
        stop = self.input(name="stop", dtype=Vec[1])

        t = Vec[self.N]

        # Counter
        q = self.output(name="q", dtype=t)
        d = self.logic(name="d", dtype=t)

        # Control
        going = self.logic(name="going", dtype=Vec[1])
        going_next = self.logic(name="going_next", dtype=Vec[1])
        going_pos = self.logic(name="going_pos", dtype=Vec[1])

        # Going set/reset
        self.expr(going_next, ITE(going, ~stop, go))
        self.dff(going, going_next, clk, rst=rst, rval="1b0")
        self.expr(going_pos, ~going & going_next)

        going_direction = self.logic(name="going_direction", dtype=Direction)
        going_direction_next = self.logic(name="going_direction_next", dtype=Direction)

        self.expr(going_direction_next, ITE(going_pos, direction, going_direction))
        self.dff(going_direction, direction, clk, en=going_pos, rst=rst, rval=Direction.RIGHT)

        def data(q: t, direction: Direction) -> Vec:
            match direction:
                case Direction.LEFT:
                    return cat(~q[-1], q[:-1])
                case Direction.RIGHT:
                    return cat(q[1:], ~q[0])
                case _:
                    return t.xprop(direction)

        self.combi(d, data, q, going_direction_next)
        self.dff(q, d, clk, en=going_next, rst=rst)


class Top(Module):
    async def drv_inputs(self):
        self.go.next = False
        self.direction.next = Direction.DC
        self.stop.next = False

        await self.rst.negedge()

        for _ in range(2):
            await self.clk.posedge()
        self.go.next = True
        self.direction.next = Direction.LEFT

        await self.clk.posedge()
        self.go.next = False
        self.direction.next = Direction.DC

        for _ in range(9):
            await self.clk.posedge()
        self.stop.next = True

        await self.clk.posedge()
        self.stop.next = False

        for _ in range(2):
            await self.clk.posedge()
        self.go.next = True
        self.direction.next = Direction.RIGHT

        await self.clk.posedge()
        self.go.next = False
        self.direction.next = Direction.DC

        for _ in range(8):
            await self.clk.posedge()
        self.stop.next = True

        await self.clk.posedge()
        self.stop.next = False

    async def mon_outputs(self):
        await self.rst.negedge()

        print("+======+===========+======+========+")
        print("|  go  | direction | stop |    q   |")
        print("+------+-----------+------+--------+")

        while True:
            await self.clk.posedge()
            dir_val = self.direction.value
            dir_str = "" if dir_val == "1b-" else dir_val.name
            print(f"|  {self.go.value} | {dir_str:^9} |  {self.stop.value} | {self.dut.q.value} |")

    def build(self):
        clk = self.logic(name="clk", dtype=Vec[1])
        rst = self.logic(name="rst", dtype=Vec[1])

        go = self.logic(name="go", dtype=Vec[1])
        direction = self.logic(name="direction", dtype=Direction)
        stop = self.logic(name="stop", dtype=Vec[1])

        self.submod(
            name="dut",
            mod=JohnsonCounter(N=4),
        ).connect(
            clk=clk,
            rst=rst,
            go=go,
            direction=direction,
            stop=stop,
        )

        # Positive clock w/ no phase shift, period T=2, 50% duty cycle
        self.drv(drv_clock(clk, shiftticks=0, onticks=1, offticks=1))

        # Positive reset asserting from T=[1..2]
        self.drv(drv_reset(rst, shiftticks=1, onticks=1))

        self.drv(self.drv_inputs())
        self.mon(self.mon_outputs())

In [ ]:
with (
    open("jc.vcd", "w") as f,
    VCDWriter(f, timescale="1ns") as vcdw,
):
    top = Top(name="top")
    top.dump_vcd(vcdw, ".*")
    run(top.main(), ticks=55)